In [1]:
# from datasets import load_dataset

import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from langchain.schema import Document
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore


import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
# ds = load_dataset("IlyaGusev/habr", trust_remote_code=True)
# ds.save_to_disk("D:\Vscode_projects\Intro_to_LLM\data\habr_data")
# df = ds["train"].to_pandas()

In [3]:
df10 = pd.read_csv('../data/habr_10_percent.csv')

In [4]:
df10.sample(3)

,id,language,url,title,text_markdown,text_html,author,original_author,original_url,lead_html,lead_markdown,type,time_published,statistics,labels,hubs,flows,tags,reading_time,format,complexity,comments
27781,264573,ru,https://habr.com/ru/post/264573/,Как не нужно строить сети,Доброго времени суток.\nНа самом деле никогда ...,"<div xmlns=""http://www.w3.org/1999/xhtml""><img...",ProotIK,NaN,NaN,"<img src=""https://habrastorage.org/getpro/habr...",Доброго времени суток.\nНа самом деле никогда ...,article,1440054392,"{'commentsCount': 14, 'favoritesCount': 465, '...",[],['itstandarts'],['develop'],['сети передачи данных'],9.0,NaN,NaN,"{'id': array([8543031, 8543039, 8543077, 85431..."
5281,431338,ru,https://habr.com/ru/post/431338/,Книга «ДНК. История генетической революции»,Познакомьтесь с историей генетической революци...,"<div xmlns=""http://www.w3.org/1999/xhtml""><a h...",ph_piter,NaN,NaN,"<a href=""https://habr.com/company/piter/blog/4...",Познакомьтесь с историей генетической революци...,article,1543395788,"{'commentsCount': 4, 'favoritesCount': 34, 're...",[],['piter' 'read' 'popular_science' 'brain'],['management' 'popsci'],['книги'],14.0,NaN,NaN,"{'id': array([19425684, 19425780, 19425828, 19..."
27017,508946,ru,https://habr.com/ru/post/508946/,"Руководство по анализу Sysmon-угроз, часть 1",Эта статья является первой частью серии по ана...,"<div xmlns=""http://www.w3.org/1999/xhtml""><img...",Лада Розенкранц (Varonis),Andy Green,https://www.varonis.com/blog/sysmon-threat-det...,"<img src=""https://habrastorage.org/webt/5z/nu/...",Эта статья является первой частью серии по ана...,article,1593520560,"{'commentsCount': 2, 'favoritesCount': 22, 're...",['translation'],['varonis' 'infosecurity' 'sys_admin' 'powersh...,['develop' 'admin'],['sysmon' 'powershell' 'microsoft' 'графы угроз'],6.0,NaN,NaN,"{'id': array([21824224, 21824682], dtype=uint6..."


In [5]:
df10['text_w_title'] = df10['title'] + "\n" +df10['text_markdown']
df10["text_w_title"] = df10["text_w_title"].astype(str)
df10["url"] = df10["url"].astype(str)
df10["title"] = df10["title"].astype(str)

## Создание векторной БД и добавление туда документов

In [6]:
separators=[
        "\n\n",             # Разделение по абзацам
        "\n",               # Перенос строки
        "[code]", "```",    # Кодовые блоки
        "####",             # Заголовки четвертого уровня
        "###",              # Заголовки третьего уровня
        "##",               # Заголовки второго уровня
        "#",                # Заголовки первого уровня
        ".",                # Точки в предложениях
        " "
    ]
def create_doc_list(df10, chunk_size=1024, chunk_overlap=128, separators=separators):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, separators=separators,
        chunk_overlap=chunk_overlap)
    
    documents = []
    for _, row in df10.iterrows():
        chunks = text_splitter.split_text(row["text_w_title"])
        for chunk in chunks:
            documents.append(Document(
                page_content=chunk,
                metadata={"link": str(row["url"]), "title": str(row["title"])}
            ))
    return documents

In [7]:
# docs = create_doc_list(df10)
# embedder = HuggingFaceEmbeddings(model_name="sergeyzh/rubert-tiny-turbo")
# client = QdrantClient(path="../data/langchain_qdrant_10percent")

# client.create_collection(
#     collection_name="embed_collection_10percent",
#     vectors_config=VectorParams(size=312, distance=Distance.COSINE),
# )

# vector_store = QdrantVectorStore(
#     client=client,
#     collection_name="embed_collection_10percent",
#     embedding=embedder,
# )

# vector_store.add_documents(docs)

### Подключение к Qdrant

In [6]:
client = QdrantClient(path="../data/langchain_qdrant_10percent")
embedder = HuggingFaceEmbeddings(model_name="sergeyzh/rubert-tiny-turbo")
collection_name = "embed_collection_10percent"

vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embedder
)

C:\Users\ivann\AppData\Local\Temp\ipykernel_16784\2378406736.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="sergeyzh/rubert-tiny-turbo")
d:\Vscode_projects\Intro_to_LLM\habr_rag_bot\.habr_rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [ ]:
query = "Хочешь в университет — сделай презентацию\nАбитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо сдачи основных экзаменов, представить четырехстраничные презентации в формате PowerPoint — на свободную тему.\nЧастично новые требования связаны с тем, что PowerPoint становится одним из бизнес-инструментов, а кроме того, считают авторы инициативы, в презентации абитуриент сможет раскрыть свои новые качества, которые не смогут проявиться на традиционных экзаменах.\nФормат PowerPoint стал своего рода языком делового общения. Ежедневно, по оценкам Microsoft, в мире показывается порядка 30 млн презентаций.\nvia AP"
results = retriever.get_relevant_documents(query)

for result in results:
    print(result)
    print("\n")

C:\Users\ivann\AppData\Local\Temp\ipykernel_16784\2087968362.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


page_content='Хочешь в университет — сделай презентацию
Абитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо сдачи основных экзаменов, представить четырехстраничные презентации в формате PowerPoint — на свободную тему.
Частично новые требования связаны с тем, что PowerPoint становится одним из бизнес-инструментов, а кроме того, считают авторы инициативы, в презентации абитуриент сможет раскрыть свои новые качества, которые не смогут проявиться на традиционных экзаменах.
Формат PowerPoint стал своего рода языком делового общения. Ежедневно, по оценкам Microsoft, в мире показывается порядка 30 млн презентаций.
via AP' metadata={'link': 'https://habr.com/ru/post/12730/', 'title': 'Хочешь в университет — сделай презентацию', '_id': '764af7ce1e064141a8f4b541ee452472', '_collection_name': 'embed_collection_10percent'}


page_content='Магистратура для стартапов
Презентация магистерской программы «Master of Technology Entrepreneurship» начнется в 18.00